In [3]:
import pandas as pd
train_data = pd.read_parquet('../data/XGBoost_data.parquet')
train_data = train_data.sort_values(by=['symbol_id', 'date_id', 'time_id'])


In [4]:
# List of responder columns
responder_columns = [col for col in train_data.columns if col.startswith('responder_')]

# Generate lag features
for responder in responder_columns:
    train_data[f'{responder}_lag_1'] = train_data.groupby('symbol_id')[responder].shift(1)


In [5]:
for responder in responder_columns:
    train_data[f'{responder}_rolling_mean_3'] = train_data.groupby('symbol_id')[responder].rolling(window=3).mean().reset_index(0, drop=True)
for responder in responder_columns:
    train_data[f'{responder}_lag_diff_1'] = train_data[responder] - train_data[f'{responder}_lag_1']
    train_data[f'{responder}_lag_ratio_1'] = train_data[responder] / (train_data[f'{responder}_lag_1'] + 1e-8)


In [10]:
print(train_data.head())
print(train_data.columns)


         date_id  time_id  symbol_id    weight  feature_00  feature_01  \
8799885      553        0          0  2.793216    3.674681    0.559137   
8799914      553        1          0  2.793216    3.655818   -0.729315   
8799943      553        2          0  2.793216    3.674681   -0.448754   
8799972      553        3          0  2.793216    3.461334   -0.521957   
8800001      553        4          0  2.793216    3.524649   -0.812735   

         feature_04  feature_05  feature_06  feature_07  ...  \
8799885   -2.866148    1.045130    1.190328   -0.050570  ...   
8799914   -2.866148    1.463785    1.666280    0.389528  ...   
8799943   -2.866148    1.694624    1.666280    1.319947  ...   
8799972   -2.547946    1.694624    1.666280    1.692438  ...   
8800001   -2.866148    1.694624    1.666280    1.692438  ...   

         responder_4_lag_diff_1  responder_4_lag_ratio_1  \
8799885                0.000000                 0.000000   
8799914               -0.633688                 0.

In [9]:
train_data.fillna(0, inplace=True)


In [ ]:
train_data.to_parquet('../data/XGBoost_data.parquet', index=False)